In [ ]:
import pickle
import h5py
import math
import numpy as np
import pandas as pd
import datetime

def create_dataset(file):
    f = h5py.File(file,'r')
    dset = f['/waves/art']
    indices=f['/waves/art'].attrs["index"][0]
    min_idx=indices[0]
    max_idx=math.floor(indices[2]/1000) * 1000
    dset=dset[min_idx:max_idx]
    reshaped_dset=np.array(dset).reshape(int((indices[2])/(indices[3]*10)), int(indices[3])*10) # because 10 second data
    df_wave = pd.DataFrame(reshaped_dset, columns=[f'feature_{i}' for i in range(1000)])
    return df_wave, indices

def predict_target(model, dataset, indices):
    # load the model from disk
    loaded_model = pickle.load(open(model, 'rb'))
    predictions=loaded_model.predict(dataset)
    dataset["predicted_annotation"] = predictions.tolist()
    dataset=pd.concat([generate_times(indices).T, dataset], axis=1)
    return dataset


def generate_times(indices):
    dt_time = datetime.datetime.fromtimestamp(indices[1]/1000000)
    return pd.Series(pd.date_range(start=dt_time, periods=int((indices[2])/(indices[3]*10)), freq=f"{1000000/indices[3]}ms"))

In [ ]:
create_dataset("../data/TBI_003.hdf5")
pre